# 使用自动化机器学习训练分类模型

有多种可供你用于训练模型的机器学习算法，有时很难确定针对特定数据和预测需求的最有效算法。 此外，通过使用标准化、缺失特征填补等技术对训练数据进行预处理，可以显著影响模型的预测性能。 在寻找满足您的要求的最佳模型时，可能需要尝试多种算法和预处理转换的组合；这需要大量的时间和计算资源。

通过 Azure 机器学习，可以使用不同的算法和预处理选项来自动比较已训练的模型。 你可以通过 [Azure 机器学习工作室](https://ml/azure.com)中的可视化界面或者 Python SDK (v2) 来使用此功能。 使用此 Python SDK，可以更好地控制自动化机器学习作业的设置，不过可视化界面更易于使用。

## 准备工作

需要最新版本的 azureml-ai-ml 包才能运行此笔记本中的代码。 运行下面的单元以验证是否已安装它。

> **注意**：
> 如果未安装“azure-ai-ml”包，请运行 `pip install azure-ai-ml` 以进行安装。

In [ ]:
## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

若要连接到工作区，我们需要标识符参数 - 订阅 ID、资源组名称和工作区名称。 已为你填写资源组名称和工作区名称。 只需订阅 ID 即可完成命令。

若要查找所需的参数，请单击工作室右上角的订阅和工作区名称。 右侧将打开一个窗格。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 复制订阅 ID，并将“YOUR-SUBSCRIPTION-ID”替换为复制的值。 </p>

## 准备数据

无需创建训练脚本来进行自动化机器学习，但需要加载训练数据。 

在本例中，你将使用包含糖尿病患者详细信息的数据集。 

若要将数据集作为输入传递至自动化机器学习作业，数据必须采用表格形式并包含目标列。 要将数据解释为表格数据集，输入数据集必须是 MLTable。

在设置过程中，系统已为你创建 MLTable 数据资产。 导航到“数据”页，可以浏览数据资产。 在此处，通过指定数据资产的名称 `diabetes-training-table` 和版本 `1`，可以检索数据资产。 

In [ ]:
## 配置自动化机器学习作业

现在可以配置自动化机器学习试验。

运行以下代码时，它将创建自动化机器学习作业，该作业：

- 使用名为 `aml-cluster` 的计算群集
- 将 `Diabetic` 设置为目标列
- 将 `accuracy` 指定为主要指标
- 总训练时长超过 `60` 分钟即会超时 
- 最多训练 `5` 个模型
- 不会使用 `LogisticRegression` 算法训练模型

In [ ]:
## 运行自动化机器学习作业

现在可以开始操作了。 现在运行自动化机器学习试验。

> 注意：这可能需要一些时间！

在作业运行时，可以在工作室中监视作业。

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named `aml-cluster`
- Sets `Diabetic` as the target column
- Sets `accuracy` as the primary metric
- Times out after `60` minutes of total training time 
- Trains a maximum of `5` models
- No model will be trained with the `LogisticRegression` algorithm

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

While the job is running, you can monitor it in the Studio.